In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.svm import SVC
import datetime

#from library.sb_utils import save_file

In [2]:
# load the feature matrix to check whether it was properly saved.

feature_matrix = pd.read_csv('../data/processed/feature_matrix_noSkip_skip1_TF_processed.csv')
feature_matrix.head().T

,0,1,2,3,4
Unnamed: 0,0,10000,20000,30000,40000
event_id,0,1,2,3,4
session_id,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e
session_position,1,2,3,4,5
skip_1,False,False,False,False,False
...,...,...,...,...,...
DIFF(tf.us_popularity_estimate) by session_id_skip_1_True,0.0,0.0,0.0,0.0,0.0
DIFF(tf.valence) by session_id,NaN,0.184898,0.03671,0.275558,0.003501
DIFF(tf.valence) by session_id_not_skipped_True,0.0,0.0,0.184898,0.03671,0.275558
DIFF(tf.valence) by session_id_skip_1_False,0.0,0.0,0.184898,0.03671,0.275558


In [3]:
feature_matrix.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('bool'), dtype('float64')],
      dtype=object)

In [4]:
feature_matrix.columns[feature_matrix.dtypes=='O']

Index(['session_id', 'session_id_skip_label', 'session_id_skip_1_False',
       'session_id_skip_1_True', 'session_id_not_skipped_True',
       'session_id_not_skipped_False'],
      dtype='object')

In [5]:
# remove the rows containing NaN, as they are the first few skipped tracks in a session
feature_matrix.drop(columns=['session_id_skip_1_False',
       'session_id_skip_1_True', 'session_id_not_skipped_True',
       'session_id_not_skipped_False','session_id_skip_label','Unnamed: 0','event_id'],inplace=True)
#feature_matrix.dropna(inplace=True)

In [6]:
# the function of integrating the skipping labels into one column
def skip_label(df):
    skip = (df['not_skipped']==False).astype(int)*4 # no skip: 0, ultra-late skip: 4
    # It has to go under this order. If skip_1 = True, then skip_2 and _3 will be True too.
    skip[df['skip_3']==True] = 3 # late skip
    skip[df['skip_2']==True] = 2 # mid skip
    skip[df['skip_1']==True] = 1 # early skip
    return skip

feature_matrix['skip_label'] = skip_label(feature_matrix)

In [7]:
feature_matrix.shape

(167880, 957)

In [8]:
feature_matrix.columns[0:50]

Index(['session_id', 'session_position', 'skip_1', 'skip_2', 'skip_3',
       'not_skipped', 'context_switch', 'no_pause_before_play',
       'short_pause_before_play', 'long_pause_before_play',
       'hist_user_behavior_is_shuffle', 'hour_of_day', 'premium',
       'hist_user_behavior_reason_start_appload',
       'hist_user_behavior_reason_start_backbtn',
       'hist_user_behavior_reason_start_clickrow',
       'hist_user_behavior_reason_start_endplay',
       'hist_user_behavior_reason_start_fwdbtn',
       'hist_user_behavior_reason_start_playbtn',
       'hist_user_behavior_reason_start_remote',
       'hist_user_behavior_reason_start_trackdone',
       'hist_user_behavior_reason_start_trackerror', 'context_type_catalog',
       'context_type_charts', 'context_type_editorial_playlist',
       'context_type_personalized_playlist', 'context_type_radio',
       'context_type_user_collection', 'euc_dist_all', 'cos_dist_all',
       'man_dist_all', 'CUM_MAX(context_switch) by session

# Simple model 3: split the beginning/end of each sequence, random forest classifier, muticlasses

This model will predict all the early and late skipping behavior.

In [ ]:
data_train_append = []
data_test_append = []
train_perc = 0.8
s_list = feature_matrix['session_id'].unique()

for s_id in s_list:
    temp = []
    temp = feature_matrix.loc[feature_matrix['session_id']==s_id,:]
    data_train_append.append(temp.iloc[0:round(len(temp)*train_perc),:])
    data_test_append.append(temp.iloc[round(len(temp)*train_perc):,:])

data_train = pd.concat(data_train_append)
data_test = pd.concat(data_test_append)

X_train = data_train.drop(columns=['not_skipped', 'skip_1', 'skip_2', 'skip_3', 'session_id','skip_label'])
X_test = data_test.drop(columns=['not_skipped', 'skip_1', 'skip_2', 'skip_3', 'session_id','skip_label'])

y_train = data_train['skip_label']
y_test = data_test['skip_label']

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(8,4), ncols=2) 

y_train.hist(ax = ax1, density = True)
ax1.set_xlabel('label')
ax1.title.set_text('training')

y_test.hist(ax = ax2)
ax2.set_xlabel('label')
ax2.title.set_text('testing')

In [ ]:
RF_MulClass = RandomForestClassifier(random_state=23, n_jobs = -1)
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
X_train2 = imp_median.fit_transform(X_train)
X_test2 = imp_median.fit_transform(X_test)

RF_MulClass.fit(X_train2, y_train)
score_MulClass_train = RF_MulClass.score(X_train2,y_train)
score_MulClass_test = RF_MulClass.score(X_test2,y_test)
print('score_train',score_MulClass_train)
print('score_test',score_MulClass_test)

In [ ]:
importances = pd.DataFrame({'FEATURE':X_train.columns,'IMPORTANCE':np.round(RF_MulClass.feature_importances_,3)})
importances = importances.sort_values('IMPORTANCE',ascending=False).set_index('FEATURE')

fig, ax = plt.subplots(figsize=(10,5)) 
sel_range = 30
sns.barplot(y = importances[0:sel_range].index, x = importances['IMPORTANCE'][0:sel_range], orient = 'h', ax = ax)

plt.tight_layout()
plt.show()

In [ ]:
y_test_pred = RF_MulClass.predict(X_test)
pl = confusion_matrix(y_test,y_test_pred)
ax= plt.subplot()
p=sns.heatmap(pl, annot=True,cmap ='Blues',ax=ax)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels');

In [44]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.74      0.64      0.69     12976
           1       0.63      0.86      0.73     13007
           2       0.21      0.09      0.12      2926
           3       0.31      0.23      0.27      3989
           4       0.26      0.05      0.09       568

    accuracy                           0.62     33466
   macro avg       0.43      0.38      0.38     33466
weighted avg       0.59      0.62      0.59     33466



**Results of Model 3:** The model performs relatively well for no skip (0) and early skip (1), which is likely because they have more cases. But in general the model performance is not so good, compared to binary classification.

# Simple model 4: split the beginning/end of each sequence, random forest classifier with hyperparameter tuning, muticlasses
Not sure whether it can still be called as a simple model, but the hyperparameter tunning will be implemented here.

In [85]:
from sklearn.metrics import make_scorer, accuracy_score, balanced_accuracy_score, roc_auc_score

param_grid = {'n_estimators':np.arange(20,200,20), 
             'max_depth':[np.arange(10,100,10),None], 
             'class_weight': ['balanced', None]}
RF_MulClass_pt = RandomForestClassifier(random_state=23, n_jobs = -1)
rf_cv= GridSearchCV(RF_MulClass_pt,
                    param_grid,
                    cv=5, 
                    n_jobs = -1, 
                    scoring=['accuracy','balanced_accuracy','f1_weighted'],
                    refit='accuracy')
rf_cv.fit(X_train,y_train)


pd.DataFrame(rf_cv.cv_results_).to_csv('../models/simpleModel/RF_GridSearchCV_20221117.csv')
from joblib import dump, load
dump(rf_cv, '../models/simpleModel/RF_GridSearchCV_20221117.joblib') 

['../models/simpleModel/RF_GridSearchCV_20221117.joblib']

In [86]:
pd.DataFrame(rf_cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_max_depth,param_n_estimators,params,split0_test_accuracy,split1_test_accuracy,...,std_test_balanced_accuracy,rank_test_balanced_accuracy,split0_test_f1_weighted,split1_test_f1_weighted,split2_test_f1_weighted,split3_test_f1_weighted,split4_test_f1_weighted,mean_test_f1_weighted,std_test_f1_weighted,rank_test_f1_weighted
0,8.878915,0.213197,0.058399,0.005612,balanced,10,20,"{'class_weight': 'balanced', 'max_depth': 10, ...",0.604434,0.598631,...,0.002998,9,0.594470,0.591604,0.596507,0.595367,0.595331,0.594656,0.001658,25
1,15.711818,0.583674,0.110991,0.030370,balanced,10,40,"{'class_weight': 'balanced', 'max_depth': 10, ...",0.605252,0.600789,...,0.004532,1,0.594636,0.592853,0.599019,0.596342,0.596066,0.595783,0.002038,19
2,25.405899,0.555974,0.185798,0.020883,balanced,10,60,"{'class_weight': 'balanced', 'max_depth': 10, ...",0.605401,0.603467,...,0.003600,7,0.594496,0.594492,0.598815,0.596527,0.596865,0.596239,0.001625,13
3,36.204568,1.373259,0.191628,0.039414,balanced,10,80,"{'class_weight': 'balanced', 'max_depth': 10, ...",0.605699,0.601867,...,0.003750,3,0.594666,0.593144,0.599369,0.595746,0.597825,0.596150,0.002217,15
4,45.903885,0.539368,0.208116,0.073984,balanced,10,100,"{'class_weight': 'balanced', 'max_depth': 10, ...",0.605401,0.600900,...,0.003912,5,0.594347,0.591873,0.598670,0.594203,0.597831,0.595385,0.002513,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,82.456485,0.967203,1.366047,0.110382,None,90,100,"{'class_weight': None, 'max_depth': 90, 'n_est...",0.611539,0.611911,...,0.001448,92,0.577610,0.577907,0.581818,0.576071,0.580364,0.578754,0.002059,81
158,98.227808,1.432475,1.532397,0.089394,None,90,120,"{'class_weight': None, 'max_depth': 90, 'n_est...",0.611799,0.611390,...,0.001378,99,0.577808,0.577181,0.581181,0.576941,0.580535,0.578729,0.001773,83
159,118.085556,1.282107,1.956185,0.077886,None,90,140,"{'class_weight': None, 'max_depth': 90, 'n_est...",0.612952,0.611130,...,0.001037,75,0.578590,0.576824,0.581511,0.576651,0.581145,0.578944,0.002065,77
160,131.518368,2.871405,2.034865,0.188263,None,90,160,"{'class_weight': None, 'max_depth': 90, 'n_est...",0.612878,0.611911,...,0.000945,81,0.578579,0.577532,0.581511,0.577304,0.580157,0.579017,0.001603,72


In [87]:
print("Best Score:" + str(rf_cv.best_score_))
print("Best Parameters: " + str(rf_cv.best_params_))

Best Score:0.6441963078173877
Best Parameters: {'class_weight': None, 'max_depth': 10, 'n_estimators': 60}


In [88]:
pd.DataFrame(rf_cv.cv_results_).columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_class_weight', 'param_max_depth', 'param_n_estimators', 'params',
       'split0_test_accuracy', 'split1_test_accuracy', 'split2_test_accuracy',
       'split3_test_accuracy', 'split4_test_accuracy', 'mean_test_accuracy',
       'std_test_accuracy', 'rank_test_accuracy',
       'split0_test_balanced_accuracy', 'split1_test_balanced_accuracy',
       'split2_test_balanced_accuracy', 'split3_test_balanced_accuracy',
       'split4_test_balanced_accuracy', 'mean_test_balanced_accuracy',
       'std_test_balanced_accuracy', 'rank_test_balanced_accuracy',
       'split0_test_f1_weighted', 'split1_test_f1_weighted',
       'split2_test_f1_weighted', 'split3_test_f1_weighted',
       'split4_test_f1_weighted', 'mean_test_f1_weighted',
       'std_test_f1_weighted', 'rank_test_f1_weighted'],
      dtype='object')

In [95]:
pd.DataFrame(rf_cv.cv_results_).sort_values('mean_test_f1_weighted')[['params','mean_test_f1_weighted','mean_test_accuracy','mean_test_balanced_accuracy']].head()

,params,mean_test_f1_weighted,mean_test_accuracy,mean_test_balanced_accuracy
45,"{'class_weight': 'balanced', 'max_depth': 60, ...",0.566791,0.598457,0.362806
72,"{'class_weight': 'balanced', 'max_depth': 90, ...",0.566860,0.598569,0.362893
63,"{'class_weight': 'balanced', 'max_depth': 80, ...",0.566860,0.598569,0.362893
54,"{'class_weight': 'balanced', 'max_depth': 70, ...",0.566860,0.598569,0.362893
36,"{'class_weight': 'balanced', 'max_depth': 50, ...",0.567050,0.598874,0.363244


**Results of Model 4:** Without feature engineering, the best parameters of the random forest model are {'class_weight': None, 'max_depth': 10, 'n_estimators': 50}, and the best accuracy was 64.4%, which is only 2% better.

# Split each sequence: multi-class (SVM)

In [31]:
# clf = make_pipeline(StandardScaler(), SVC(verbose = True, random_state=23))
# clf.fit(X_train, y_train)

[LibSVM]..............................................................................................................................*..............................................................................................................................*.*
optimization finished, #iter = 252227
obj = -36073.929342, rho = 0.292277
nSV = 47263, nBSV = 30604
.................................................*................................*
optimization finished, #iter = 81440
obj = -23087.018105, rho = -0.397195
nSV = 28683, nBSV = 21009
.................................................................*...........................................*.*
optimization finished, #iter = 108221
obj = -29105.703096, rho = -0.295036
nSV = 35559, nBSV = 26185
......................................................................................*.........................................................................................*.*
optimization finished, #iter = 176460
obj = -22031.591988

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(random_state=23, verbose=True))])

In [32]:
# score_clf_train = clf.score(X_train,y_train)
# score_clf_test = clf.score(X_test,y_test)
# print('score_train',score_clf_train)
# print('score_test',score_clf_test)

score_train 0.6678768580653801
score_test 0.6515568039203968


In [33]:
# from joblib import dump, load
# dump(clf, '../models/simpleModel/SVC_simple.joblib') 

['../models/simpleModel/SVC_simple.joblib']